**Task 07: Querying RDF(s)**

In [21]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Read the RDF file

In [22]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Na71f688852a34ef39bfd53758784149f (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

First, we show all the triples in the graph:

In [23]:
for s, p, o in g:
  print(s,p,o)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#UPM http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#University
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
http://somewhere#University http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#JaneSmith http://somewhere#htt

If we ask directly for the subclasses of Person, we won't obtain the ones that are subclass of one of these subclasses. Let's see all the triples with the property "subClassOf":

In [24]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")

for s,p,o in g.triples((None, RDFS.subClassOf, None)):
  print(s, p, o)

http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person


We can see that both Researcher and PhDstudent are subclass of Person, due to the transitivity of the relation. Now, we can solve the problem:

In [25]:
# Using RDFLib (Iterating over the graph)
class_person = ns.Person
for t in g.triples((None, RDFS.subClassOf, class_person)): # Ask for the direct subclass
  print(t[0])
  class_person = t[0]
  for t in g.triples((None, RDFS.subClassOf, class_person)): # Ask for the subclasses of the last subclass
      print(t[0])
      class_person = t[0]
     

http://somewhere#Researcher
http://somewhere#PhDstudent


In [26]:
# Using SPARQL
q1 = prepareQuery('''
  SELECT 
    ?subClass
  WHERE {
    ?subClass RDFS:subClassOf* ns:Person.
    FILTER (?subClass != ns:Person).

  }  
  ''',
  initNs = { "ns": ns, "RDFS" : RDFS}
)

for r in g.query(q1):
  print(r.subClass)

http://somewhere#Researcher
http://somewhere#PhDstudent


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


Let's see all the instances

In [27]:
for s,p,o in g.triples((None, RDF.type, None)):
  print(s, p, o)

http://somewhere#PhDstudent http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#Researcher http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#UPM http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#University
http://somewhere#University http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org

We can see that there are three inidviduals of Person, two of then are instances of Person and one, instance of Research.

In [28]:
# Using RDFLib (Iterating over the graph)
class_person = ns.Person
for t in g.triples((None, RDF.type, class_person)): # Instances of Person
  print(t[0])

for t in g.triples((None, RDFS.subClassOf, class_person)): # Instances of direct subclasses of Person
  class_person = t[0]
  for t in g.triples((None, RDF.type, class_person)):
      print(t[0])

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


In [29]:
# Using SPARQL
q2 = prepareQuery('''
  SELECT 
    ?instance
  WHERE {
    ?subClass RDFS:subClassOf* ns:Person.
    ?instance a ?subClass.

  }  
  ''',
  initNs = { "ns": ns, "RDFS" : RDFS}
)

for r in g.query(q2):
  print(r.instance)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [30]:
# Using RDFLib (Iterating over the graph)

# First, obtain the individual

class_person = ns.Person
individual = []
for t in g.triples((None, RDF.type, class_person)): # Instances of Person
  individual.append([t[0], class_person])

for t in g.triples((None, RDFS.subClassOf, class_person)): # Instances of direct subclasses of Person
  class_person = t[0]
  for t in g.triples((None, RDF.type, class_person)):
      individual.append([t[0], class_person])

# Second, look for the properties

for i in individual:
  print("Instance " + str(i[0]) + ", Class " + str(i[1]))
  for s,p,o in g.triples((i[0], None, None)):
    print("- Property: " + str(p) + ", Value: " + str(o))


Instance http://somewhere#SaraJones, Class http://somewhere#Person
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type, Value: http://somewhere#Person
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN, Value: Sara Jones
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family, Value: Jones
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given, Value: Sara
Instance http://somewhere#JohnSmith, Class http://somewhere#Person
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN, Value: John Smith
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given, Value: John
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type, Value: http://somewhere#Person
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family, Value: Smith
Instance http://somewhere#JaneSmith, Class http://somewhere#Researcher
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given, Value: Jane
- Prop

In [31]:
# Using SPARQL
q3 = prepareQuery('''
  SELECT 
    ?instance ?subClass
  WHERE {
    ?subClass RDFS:subClassOf* ns:Person.
    ?instance a ?subClass.

  }  
  ''',
  initNs = { "ns": ns, "RDFS" : RDFS}
)

q4 = prepareQuery('''
  SELECT 
    ?property ?value
  WHERE {
    ?instance ?property ?value

  }  
  ''',
  initNs = { "ns": ns, "RDFS" : RDFS}
)

for r in g.query(q3):
  print("Instance " + str(r.instance) + ", Class " + str(r.subClass))
  for p,o in g.query(q4, initBindings = {'?instance' : r.instance}):
    print("- Property: " + str(p) + ", Value: " + str(o))

Instance http://somewhere#SaraJones, Class http://somewhere#Person
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type, Value: http://somewhere#Person
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN, Value: Sara Jones
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family, Value: Jones
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given, Value: Sara
Instance http://somewhere#JohnSmith, Class http://somewhere#Person
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN, Value: John Smith
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given, Value: John
- Property: http://www.w3.org/1999/02/22-rdf-syntax-ns#type, Value: http://somewhere#Person
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family, Value: Smith
Instance http://somewhere#JaneSmith, Class http://somewhere#Researcher
- Property: http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given, Value: Jane
- Prop